In [ ]:
!pip install pandas
!pip install rdkit
!pip install git+https://github.com/mcs07/PubChemPy.git@a5d8646

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem.Draw import MolsToGridImage
from rdkit.Chem import MolFromSmiles, rdFMCS, Draw, MolFromSmarts, MolToSmiles
import pubchempy as pcp

In [ ]:
data_smiles = pd.read_csv('https://raw.githubusercontent.com/sharonlae77/useful_codes/main/data/data_smiles.csv')

molecule = [MolFromSmiles(i) for i in data_smiles['smiles']]

res = rdFMCS.FindMCS(molecule,
                     completeRingsOnly=True
                    )

substructure_smarts = res.smartsString

to_highlight = []

for mol in molecule:
  substructure = Chem.MolFromSmarts(substructure_smarts)
  if mol and substructure:
    match = mol.GetSubstructMatch(substructure)
    if match:
      to_highlight.append(match)
    else:
      to_highlight.append([])
  else:
    to_highlight.append([])

legends = []
for i in range(len(data_smiles)):
    legend = data_smiles['name'].iloc[i]
    legends.append(legend)

opts = Draw.MolDrawOptions()
img = MolsToGridImage(molecule,
											molsPerRow=2,
											subImgSize=(1000, 600),
											legends=legends,
											returnPNG=True,
                      drawOptions=opts,
                      highlightAtomLists=to_highlight)
img

In [ ]:
substructure_smiles = MolToSmiles(MolFromSmarts(substructure_smarts))
compounds = pcp.get_compounds(substructure_smiles, namespace='smiles')
if compounds:
    match = compounds[0]
    print(match.iupac_name)
else:
    print("Compound not found.")